In [1]:
# data_cleaning.ipynb
# Ce notebook charge le CSV brut, nettoie les données patient et labels,
# vérifie la présence des images, crée des colonnes binaires pour les pathologies,
# et sauvegarde un CSV propre (cleaned_data.csv) pour la suite du projet.
import os
import pandas as pd
import logging

In [3]:
# --- Configuration Logging ---
os.makedirs('../logs', exist_ok=True)
logging.basicConfig(
    filename='../logs/image_errors.log',
    level=logging.WARNING,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [4]:
# --- Définition des chemins ---
data_path = '../data/raw'
raw_csv = os.path.join(data_path, 'data_Entry_2017.csv')
processed_dir = '../data/processed'
os.makedirs(processed_dir, exist_ok=True)
clean_csv = os.path.join(processed_dir, 'cleaned_data.csv')
images_dir = os.path.join(data_path, 'images')


In [5]:
# --- Chargement du CSV ---
try:
    df = pd.read_csv(raw_csv)
    print(f"CSV chargé avec succès ({len(df)} lignes)")
except Exception as e:
    logging.error(f"Erreur lors du chargement du CSV : {e}")
    raise

# --- Nettoyage des colonnes ---
df.columns = df.columns.str.strip()
df = df[['Image Index', 'Finding Labels', 'Patient Age', 'Patient Gender', 'Patient ID']]

# --- Suppression des doublons ---
df.drop_duplicates(subset=['Image Index'], inplace=True)

# --- Suppression des valeurs manquantes ---
df.dropna(subset=['Finding Labels', 'Patient ID'], inplace=True)

# --- Correction des types ---
df['Patient Age'] = pd.to_numeric(df['Patient Age'], errors='coerce')
df['Patient Gender'] = df['Patient Gender'].astype(str).str.upper().str.strip()

# --- Filtrage logique ---
df = df[(df['Patient Age'] > 0) & (df['Patient Age'] < 120)]
df = df[df['Patient Gender'].isin(['M', 'F'])]

# --- Nettoyage des labels ---
df['Finding Labels'] = df['Finding Labels'].str.replace(r'No Finding', 'Normal', regex=True)
df['Finding Labels'] = df['Finding Labels'].str.replace(r'\|+', '|', regex=True)
df['Finding Labels'] = df['Finding Labels'].str.strip()

# --- Vérification des images existantes ---
existing_files = set(os.listdir(images_dir))
missing_images = set(df['Image Index']) - existing_files
if missing_images:
    logging.warning(f"Images manquantes : {len(missing_images)} fichiers")
    for img in list(missing_images)[:10]:
        logging.warning(f" - {img}")
df = df[df['Image Index'].isin(existing_files)]

# --- Conversion des labels en colonnes binaires ---
pathologies = ['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration',
               'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax', 'Consolidation',
               'Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia', 'Normal']

for p in pathologies:
    df[p] = df['Finding Labels'].apply(lambda x: 1 if p in x else 0)

# Supprimer les pathologies rares (<20 images)
rare_diseases = [p for p in pathologies if df[p].sum() < 20]
if rare_diseases:
    df.drop(columns=rare_diseases, inplace=True)
    pathologies = [p for p in pathologies if p not in rare_diseases]
    print(f"Pathologies rares supprimées : {rare_diseases}")


CSV chargé avec succès (112120 lignes)


In [6]:

print("Nettoyage terminé :")
print(f"- Nombre total d’entrées : {len(df)}")
print(f"- Âge moyen : {df['Patient Age'].mean():.2f}")
print(f"- Répartition du genre :\n{df['Patient Gender'].value_counts()}")
print(f"- Pathologies conservées : {pathologies}")


Nettoyage terminé :
- Nombre total d’entrées : 4999
- Âge moyen : 51.81
- Répartition du genre :
Patient Gender
M    2608
F    2391
Name: count, dtype: int64
- Pathologies conservées : ['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax', 'Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia', 'Normal']


In [7]:
# --- Sauvegarde du CSV nettoyé ---
df.to_csv(clean_csv, index=False)
print(f"CSV nettoyé sauvegardé : {clean_csv}")

CSV nettoyé sauvegardé : ../data/processed\cleaned_data.csv
